
# Django + UWSGI + Nginx 服务器部署

总体流程如下：

![](dun.jpg)

## 使用 Nginx 以及 Python 环境配置

Nginx 用来进行HTTP通信，在我们的系统中用来进行HTTP通信、静态文件处理以及SOCKET转发（动态处理转发给 UWSGI，然后使用 Django 进行处理）

```bash

#安装 NGINX 服务器，以处理静态文档和发送HTTP请求到UWSGI

apt-get install nginx


#创建一个虚拟环境，运行网站后端

可以使用 virtualenv

python -V

pip install virtualenv 

virtualenv cmsite

source cmsite/bin/activate

[cmsite]pip install django

[cmsite]pip install mezzanine


或者使用 conda：

conda env list

conda create cmsite python=3.6

activate cmsite(windows)/source activate cmsite(linux)

[cmsite]conda install django

[cmsite]conda install mezzanine
```

nginx 使用 `/etc/init.d/nginx restart` 来重启，相当于 `net service nginx restart`

## 使用 MEZZANINE OR Django

mezzanine 是一个 CMS 系统，基于 django。创建项目，连接数据库，然后运行测试服务器，看看是否可以正常工作。

```bash
# 用来部署到Nginx的模块
mezzanine-project project_name 

python manage.py createdb

#如果使用mysql数据库，修改./project_namee/local_setting.py。

python manage.py collectstatic

vi project_name/setting.py 
#修改允许的HOST，字段为list类型

python manage.py runserver  0.0.0.0:5800
```

如果只使用 django，为了部署，需要收集所有静态文件，并且关闭调试模式，大开允许访问的地址。

```bash
#DJANGO APP setting.py
STATIC_ROOT = os.path.join(BASE_DIR, "static/")
DEBUG = False
ALLOWED_HOSTS = ['*']

python manage.py collectstatic
```

## 使用 UWSGI

### 测试 USWGI

这一步需要确认 uwsgi -> django/mezzanine 是否可以工作

```bash
[cmsite]pip install uwsgi

uwsgi --version

测试模块状态

#DJANGO APP test.py

def application(env,start_response):
    start_response('200 OK',[('Content-Type','text/html')])
    return [b"Hello World"]

uwsgi --http :8000 --wsgi-file test.py --plugin python

对于 mezzanine/django 来说，直接这样加载动态文件：

uwsgi --http :port --module django_project_name.wsgi --plugin python
# 仅动态可被加载，静态文件并没有被加载
```
### 配置 USWGI

更普遍的，使用ini文件来加载模块，连接django项目：

uwsgi的配置如下（放在Django项目目录下的uwsgi.ini文件中）：

```python
# Django-related settings
socket = :8001 or http = :8001
# the base directory (full path)
chdir           = /home/ubuntu/blog
# Django s wsgi file
module          = blog.wsgi
# process-related settings
# master
master          = true
# maximum number of worker processes
processes       = 4
# ... with appropriate permissions - may be needed
# chmod-socket    = 664
# clear environment on exit
vacuum          = true
```

在shell中执行 `sudo uwsgi --ini uwsgi.ini ` 即可运行。

注意，可以让 uswgi 在 nohup 后台执行，如果不需要静态处理的话，这样就可以直接上线了。

`nohup uwsgi --ini /cmsite/source/uwsgi.ini >> quicklink.log 2>&1 &`

## uwsgi 和 nginx 配置连接

nginx 默认读取 `/etc/nginx/sites-enabled/default` 目录下的配置，修改 nginx.conf 如下：

```bash
server {
    # the port your site will be served on
    listen      80;
    # the domain name it will serve for
    server_name 127.0.0.1; # substitute your machine's IP address or FQDN
    charset     utf-8;

    # max upload size
    client_max_body_size 75M;   # adjust to taste

    # Django media
    location /media  {
        alias /home/ubuntu/blog/media;  # your Django project's media files - amend as required
    }

    location /static {
        alias /home/ubuntu/blog/static; # your Django project's static files - amend as required
    }

    # Finally, send all non-media requests to the Django server.
    location / {
        include     uwsgi_params; # the uwsgi_params file you installed
        uwsgi_pass 127.0.0.1:8001;
    }
}
```
这个文件告诉 nginx 接受何处的HTTP请求，到哪里去找 django 的静态和媒体文件用来处理和返回，以及协同 uwsgi 处理动态文件（要求一个对照列表和uwsgi工作的端口）

因为 django 包含源码，因此不能暴露在 nginx 的 /var/www/目录下。

uwsgi_params 放在 nginx.conf 同名目录下。

nginx 使用 /etc/init.d/nginx restart 来重启服务，读取配置。


## 后续

/etc/rc.local 中 放置 uwsgi 的执行命令以自动开机执行。或者使用 `nohup uwsgi --ini /cmsite/source/uwsgi.ini >> quicklink.log 2>&1 &` 放置在后台执行。

